# Lab 6

Here are the  classes from lecture 16. For this lab, you will use these classes for this lab and are encouraged to modify these classes as you need.

In [1]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name
    

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [2]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        #data.__init__(self,name+" Grade Algorithm")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print self.name()+" Error: Bad Grade."
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        #data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists."
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print self.__grades[grade]
    


In [3]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError


In [4]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        #data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print self.name()+" Error: Did not find student."
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)
        
    def mean_std_print(self,mean=None,std=None):
        self.mean=mean
        self.std=std
        print ('mean \t|\t std'+'\n'
               '________|___________________')
        for i in range (0,len(mean)):
            print str(round(mean[i],4))+'\t|',str(std[i])
        
    

In [5]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().iteritems():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print self.name()+ " Error: Did not get a numerical grade as input."
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                #print percent, i, v
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

In [6]:
def mean(x):
    return sum(x)/float(len(x))

def std(x):
    variance=sum(map(lambda y:y**2, x))/float(len(x))-mean(x)**2
    return variance**(0.5)

class mean_std_calculator(calculator):
    def __init__(self,grade=None,mean_val=None,std_val=None):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        self.grade=grade
        self.mean_val=mean_val
        self.std_val=std_val
        
    def apply(self,grade_name):
        self.mean_val=mean(self.grade)
        self.std_val=std(self.grade)
        return self.mean_val,self.std_val

## CSV Reader

*Exercise 1*: The data for *your* grades in this course are stored (anonymized) in a "camma separated values" (CSV) file name `Data1401-Grades.csv` in the directory of this lab. You can see the contents using the `cat` shell command:

In [7]:
!cat Data1401-Grades.csv 

l1_n,l1_1,l2_n,l2_1,l2_2,l2_3,l2_4,l2_5,l2_6,l2_7,l3_n,l3_1,l3_2,l3_3,l3_4,l3_5,l3_6,l3_7,l3_8,l3_9,l3_10,l3_11,l3_12,l3_13,l3_14,l4_n,l4_1,l4_2,l4_3,l4_4,l4_5,l4_6,l4_7,l4_8,l4_9,l4_10,l4_11,q1_n,q1_1,e1_n,e1_1,e1_2,e1_3,e1_4,e1_5,e1_6,e1_7,e1_8,e1_9,e1_10,e1_11,e1_12,e1_13,e1_14,e1_15
1,10,7,0,10,10,8,10,10,10,14,9,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,9.5,15,9,9,0,9,8,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,9,10,10,10,7,10,3,6,3,3,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,5,15,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0
1,10,7,10,10,3,9.5,10,10,9.5,14,10,10,10,8,5,10,5,10,3,0,10,3,10,8,11,10,10,10,10,10,10,0,0,10,5,0,1,10,15,9,9,10,9,7,9,0,0,10,10,9,5,10,8,10
1,10,7,10,10,9.5,0,10,10,0,14,9.5,0,0,10,0,10,5,10,7,0,10,6,10,0,11,10,10,6,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,5,0,7,0,3,3,3,0,3,0,0
1,10,7,10,10,10,9.5,10,10,9.5,14,5,9.5,9.5,8,10,10,8,10,8,0,5,6,0,0,11,0,10,10,10,0,5,0,

You will note that the first line has the names of the "columns" of data, and that subsequent lines (or "rows") have the data for each student, separated by cammas.

Recalling that in lecture 7 we created a simple CSV file reader, create a CSV reader function that takes a filename as input and returns data structure(s) that store the data in the file. Note that you are not allowed to use a library. The point here is for *you* to write the CSV reader. Some options for your data structures (pick one):

* A list of dictionaries, where each element of the list is corresponds to a row of data and the dictionaries are keyed by the column name. For example `data[5]["l3_5"]` corresponds to the 6th student's grade on lab 3 question 5.

* A list of lists (i.e. a 2-D array or matrix) and a dictionary, where each element of the "matrix" corresponds to a a specific grade for a specific student and the dictionary maps the name of the column to the column index. For example `data[5][column_names["l1_5"]]` corresponds to the 6th student's grade on lab 3 question 5.

* A dictionary of lists, where each element of the dictionary corresponds to a column of data and the lists contain the data in that column. For example `data["l3_5"][5]` corresponds to the 6th student's grade on lab 3 question 5.

In [8]:
# You solution here.
def cvs_reader(filename):
    data=list() # if you choose first option
    contents = open(filename).read().splitlines()
    hold=list()
    for i in range(0,len(contents)):
        hold.append(contents[i].split(","))

    for i in range(0,len(hold)):
        try:
            data.append(dict(zip(hold[0],hold[i+1])))
        except(IndexError):
            break;

        
    return data

In [9]:
data=cvs_reader("Data1401-Grades.csv")
print data[0],'\n'
print data[0]['l2_4']

{'q1_n': '1', 'l3_n': '14', 'l4_n': '11', 'e1_15': '0', 'e1_14': '0', 'e1_13': '0', 'e1_12': '0', 'e1_11': '0', 'e1_10': '0', 'l4_1': '0', 'q1_1': '9.5', 'l2_n': '7', 'l4_11': '0', 'l4_10': '0', 'l4_2': '0', 'l4_5': '0', 'l1_n': '1', 'l4_4': '0', 'e1_9': '0', 'e1_8': '0', 'l4_7': '0', 'e1_3': '0', 'e1_2': '9', 'e1_1': '9', 'l4_6': '0', 'e1_7': '0', 'e1_6': '0', 'e1_5': '8', 'e1_4': '9', 'l2_3': '10', 'l2_2': '10', 'l2_1': '0', 'l2_7': '10', 'l2_6': '10', 'l2_5': '10', 'l2_4': '8', 'l4_9': '0', 'l4_8': '0', 'l3_8': '0', 'l3_9': '0', 'l3_2': '0', 'l3_3': '0', 'l4_3': '0', 'l3_1': '9', 'l3_6': '0', 'l3_7': '0', 'l3_4': '0', 'l3_5': '0', 'e1_n': '15', 'l3_10': '0', 'l3_11': '0', 'l3_12': '0', 'l3_13': '0', 'l3_14': '0', 'l1_1': '10'} 

8


## Creating a Gradebook

*Exercise 2:* In lectures 15 and 16 we used pandas to read the CSV file and create the grade book. Modify the code below to use your CSV reader instead.

In [10]:
import pandas as pd

data=cvs_reader("Data1401-Grades.csv")

a_grade_book=grade_book("Data 1401")

for student_i in range(len(data)):
    a_student_0=student("Student",str(student_i),student_i)

    for k in data[0].keys():
        #print k
        #print data[student_i][k]
        a_student_0.add_grade(grade(k,value=float(data[student_i][k])))

    a_grade_book.add_student(a_student_0)
#a_student_0.print_grades(),'\n\n'
    
    
#a_grade_book.apply_calculator(uncurved_letter_grade_percent("l2_2",max_grade=10))
for k,a_student in a_grade_book.get_students().iteritems():
    print a_student.id_number(),a_student["l2_2"]

0 l2_2: 10.0
1 l2_2: 0
2 l2_2: 0
3 l2_2: 10.0
4 l2_2: 10.0
5 l2_2: 10.0
6 l2_2: 10.0
7 l2_2: 10.0
8 l2_2: 10.0
9 l2_2: 10.0
10 l2_2: 10.0
11 l2_2: 10.0
12 l2_2: 10.0
13 l2_2: 10.0
14 l2_2: 10.0
15 l2_2: 10.0


In [11]:
#print data[0].keys()

## Grade Summing

*Exercise 3:* In lecture 16 we changed the design of our algorithms classes and then updated the `uncurved_letter_grade_percent` calculator. In lecture 15 we create a `grade_summer` calcuator that takes a prefix (for example `e1_` and a number `n`) and sums all grades starting with that prefix up to `n` and creates a new sum grade. Update this calculator (below) to the new design of our algorithm classes. Test your updated calculator by using it to sum the grades for all labs, quizzes, and exams of each students.



In [29]:
class grade_summer(calculator):
    def __init__(self,prefix,data):
        self.__prefix=prefix
        #self.__n=n
        self.__data=data
        #statistics_calculator.__init__(self,"Sum Grades")
        
    def apply(self):
        n=0
        try:
            for l in range(1,20):
                data[0][self.__prefix+str(l)+'_n']
                n+=1
                #print n
        except KeyError:
            h="hi"
        #print n    
        
        if n==1:
            boop=[self.__prefix+'1_']
        else:
            boop=[self.__prefix+str(y)+'_' for y in range(1,n+1)]
            
        labels=list()
        for y in range(0,len(boop)):
            labels.append([boop[y]+str(x) for x in range(1,int(data[0][boop[y]+'n'])+1)])
        #print labels
        
        x=0
        grade_sum= [[0] * int(len(self.__data)) for p in range(n)]
        for k in range (0,int(len(self.__data))):
            for i in range (0,len(labels)):
                for j in range (0,int(data[0][boop[i]+'n'])):
                    x+=float(data[k][labels[i][j]])
                    #print x
                grade_sum[i][k]=x
                x=0
            #a_student.add_grade(grade(prefix+"sum",value=grade_sum))
        return grade_sum

In [30]:
#len(data[0])
#print data[0]
#grade_sum=[[0] * 16 for p in range(4)]
#print grade_sum

In [31]:
labs=grade_summer('l',data)
quizzes=grade_summer('q',data)
exams=grade_summer('e',data)
########Each column corresponds to a student, same person for each column in the three variables##########
Lab_grades=labs.apply() ### each row is a different lab and each column is the student
Quiz_grades=quizzes.apply()
Exam_grades=exams.apply()
print Lab_grades,'\n',Quiz_grades,'\n',Exam_grades

[[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], [58.0, 0.0, 0.0, 62.0, 49.5, 69.0, 54.5, 69.0, 39.5, 60.0, 47.0, 68.5, 64.0, 49.5, 68.0, 59.0], [9.0, 0.0, 71.0, 102.0, 77.5, 89.0, 64.5, 40.0, 40.0, 26.0, 126.0, 97.5, 115.0, 114.5, 95.0, 106.0], [0.0, 0.0, 0.0, 75.0, 26.0, 35.0, 30.0, 51.0, 68.0, 27.0, 105.0, 51.0, 0.0, 25.0, 0.0, 53.0]] 
[[9.5, 0.0, 5.0, 10.0, 0.0, 9.5, 10.0, 10.0, 9.5, 9.5, 0.0, 10.0, 10.0, 0.0, 0.0, 9.5]] 
[[35.0, 0.0, 20.0, 115.0, 24.0, 95.0, 37.0, 77.0, 103.0, 117.0, 138.0, 110.0, 104.0, 111.0, 103.0, 134.0]]


## Curving Grades

*Exercise 4:* Use the `mean_std_calculator` above to calculate the mean and standard deviation for every lab, quiz, and exam in the class. Add a new print function to the `grade_book` class to print out such information in a nice way, and use this function to show your results.


In [32]:
# Your solution here
lab_mean=[[0] * 1 for p in range(len(Lab_grades))]
lab_std=[[0] * 1 for p in range(len(Lab_grades))]
for i in range (0,4):
    lab=mean_std_calculator(Lab_grades[i])
    lab_mean[i],lab_std[i]=lab.apply('lab'+str(i+1))
       
quiz_mean=[[0] * 1 for p in range(1)]
quiz_std=[[0] * 1 for p in range(1)]

quiz=mean_std_calculator(Quiz_grades[0])
quiz_mean[0],quiz_std[0]=quiz.apply('lab'+str(i+1)) 


exam_mean=[[0] * 1 for p in range(1)]
exam_std=[[0] * 1 for p in range(1)]
exam=mean_std_calculator(Exam_grades[0])
exam_mean[0],exam_std[0]=exam.apply('lab'+str(i+1))

#print lab_mean,lab_std,'\n',quiz_mean,quiz_std,'\n',exam_mean,exam_std

In [33]:
x=grade_book("Data1401-Grades")
print "Quiz means and standard deviations"
x.mean_std_print(quiz_mean,quiz_std)
print "\n\n"
print "Exam means and standard deviations"
x.mean_std_print(exam_mean,exam_std)
print"\n\n"
print "Lab means and standard deviations"
x.mean_std_print(lab_mean,lab_std)

Quiz means and standard deviations
mean 	|	 std
________|___________________
6.4063	| 4.46940554632



Exam means and standard deviations
mean 	|	 std
________|___________________
82.6875	| 42.9370451213



Lab means and standard deviations
mean 	|	 std
________|___________________
10.0	| 0.0
51.0938	| 21.0566340125
73.3125	| 38.3017929574
34.125	| 30.4217746852


*Exercise 5:* In lecture 16 we changed the design of our algorithms classes and then updated the `uncurved_letter_grade_percent` calculator. Do the same for the `curved_letter_grade` calculator below and by curving all the lab, quiz, and exam grades.

In [17]:
class curved_letter_grade(calculator):
    #__grades_definition=[ (.97,"A+"),
                         # (.93,"A"),
                         # (.9,"A-"),
                         # (.87,"B+"),
                         # (.83,"B"),
                         # (.8,"B-"),
                         # (.77,"C+"),
                          #(.73,"C"),
                          #(.7,"C-"),
                          #(.67,"C+"),
                          #(.63,"C"),
                          #(.6,"C-"),
                          #(.57,"F+"),
                          #(.53,"F"),
                          #(0.,"F-")]
    __max_grade=100.
    
    def __init__(self,grade,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self. __grades_definitions=[ [self.__mean+(1.333*self.__std),"A+"],
                          [self.__mean+self.__std,"A"],
                          [self.__mean+(.666*self.__std),"A-"],
                          [self.__mean+(.333*self.__std),"B+"],
                          [self.__mean,"B"],
                          [self.__mean-(.333*self.__std),"B-"],
                          [self.__mean-(.666*self.__std),"C+"],
                          [self.__mean-self.__std,"C"],
                          [self.__mean-(1.333*self.__std),"C-"],
                          [self.__mean-(1.666*self.__std),"C+"],
                          [self.__mean-(2*self.__std),"C"],
                          [self.__mean-(2.333*self.__std),"C-"],
                          [self.__mean-(2.666*self.__std),"F+"],
                          [self.__mean-(3*self.__std),"F"],
                          [self.__mean-(3.333*self.__std),"F-"]]
        self.__grade=grade
        for i in range(0,len(self. __grades_definitions)):
                    if self. __grades_definitions[i][0]>=self.__max_grade:
                        self. __grades_definitions[i][0]=self.__max_grade
        #print self. __grades_definitions
        #print self.__grade
        #calculator.__init__(self,
                                #  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                #  " Mean="+str(self.__mean)+\
                                #  " STD="+str(self.__std)+\
                                 # " Max="+str(self.__max_grade))
        

    def apply(self):
        grade_letters=[0] * len(self.__grade)
        if self.__std==0:
            for j in range(len(self.__grade)):
                grade_letters[j]=self. __grades_definitions[0][1]
        else:
            for j in range(len(self.__grade)):
                for i in range(0,len(self. __grades_definitions)):
                    if self.__grade[j]>=self. __grades_definitions[i][0]:
                        grade_letters[j]=self. __grades_definitions[i][1]
                        break
        return grade_letters  

In [18]:
Lab_letters=list()
Quiz_letters=list()
Exam_letters=list()
for j in range(len(lab_mean)):
    y=curved_letter_grade(Lab_grades[j],lab_mean[j],lab_std[j],int(data[0]['l'+str((j+1))+'_n'])*10)
    Lab_letters.append(y.apply())
    
y=curved_letter_grade(Quiz_grades[0],quiz_mean[0],quiz_std[0],int(data[0]['q'+str(1)+'_n'])*10)
Quiz_letters.append(y.apply())

y=curved_letter_grade(Exam_grades[0],exam_mean[0],exam_std[0],int(data[0]['e'+str(1)+'_n'])*10)
Exam_letters.append(y.apply())

print Lab_letters
print Lab_grades,'\n\n'


print Quiz_letters
print Quiz_grades,'\n\n'


print Exam_letters
print Quiz_grades,'\n\n'

[['A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'A+'], ['B', 'F+', 'F+', 'B+', 'B-', 'A-', 'B', 'A-', 'C+', 'B+', 'B-', 'A-', 'B+', 'B-', 'A-', 'B+'], ['C', 'C', 'B-', 'A-', 'B', 'B+', 'B-', 'C', 'C', 'C-', 'A+', 'B+', 'A', 'A', 'B+', 'A-'], ['C-', 'C-', 'C-', 'A+', 'B-', 'B', 'B-', 'B+', 'A', 'B-', 'A+', 'B+', 'C-', 'B-', 'C-', 'B+']]
[[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], [58.0, 0.0, 0.0, 62.0, 49.5, 69.0, 54.5, 69.0, 39.5, 60.0, 47.0, 68.5, 64.0, 49.5, 68.0, 59.0], [9.0, 0.0, 71.0, 102.0, 77.5, 89.0, 64.5, 40.0, 40.0, 26.0, 126.0, 97.5, 115.0, 114.5, 95.0, 106.0], [0.0, 0.0, 0.0, 75.0, 26.0, 35.0, 30.0, 51.0, 68.0, 27.0, 105.0, 51.0, 0.0, 25.0, 0.0, 53.0]] 


[['A-', 'C+', 'B-', 'A+', 'C+', 'A-', 'A+', 'A+', 'A-', 'A-', 'C+', 'A+', 'A+', 'C+', 'C+', 'A-']]
[[9.5, 0.0, 5.0, 10.0, 0.0, 9.5, 10.0, 10.0, 9.5, 9.5, 0.0, 10.0, 10.0, 0.0, 0.0, 9.5]] 


[['C-', 'C', 'C+', 'A-', 'C+', 'B',

In [19]:
print Quiz_grades[0]
print Lab_grades[0]

[9.5, 0.0, 5.0, 10.0, 0.0, 9.5, 10.0, 10.0, 9.5, 9.5, 0.0, 10.0, 10.0, 0.0, 0.0, 9.5]
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]


## Final Course Grade

*Exercise 6:* Write a new calculator that sums grades with a prefix, as in the `grade_summer` calculator, but drops `n` lowest grades. Apply the algorithm to drop the lowest lab grade in the data.


In [20]:
# Solution here
class grade_summer_drop(calculator):
    def __init__(self,prefix,data):
        self.__prefix=prefix
        #self.__n=n
        self.__data=data
        #statistics_calculator.__init__(self,"Sum Grades")
        
        
    def drop_lab(self,all_lab_grades,number_of_grades_to_drop=0):
        import copy
        self.m=number_of_grades_to_drop
        self.lab_grades=copy.deepcopy(all_lab_grades)
        hold=10000
        hold_i=0
        hold_i=0
        for k in range(self.m):
            for i in range(len(self.lab_grades[0])):
                for j in range(len(self.lab_grades)):
                    #print "went through",self.lab_grades[j][i]
                    if isinstance(self.lab_grades[j][i], float):
                        if (self.lab_grades[j][i])/float(self.__data[0]['l'+str((j+1))+'_n']*10)<hold:
                            hold=self.lab_grades[j][i]
                            #print hold,(self.lab_grades[j][i])/(float(self.__data[0]['l'+str((j+1))+'_n'])*10.0)
                            #print self.lab_grades[j][i]
                            hold_i=i
                            hold_j=j
                #print hold_i,hold_j
                self.lab_grades[hold_j][hold_i]='dropped'
                hold=10000
        return self.lab_grades
    
    
    
    def apply(self,number_of_grades_to_drop=0): 
################ code from exercise 3 ######################################
        m=number_of_grades_to_drop
        n=0
        try:
            for l in range(1,20):
                data[0][self.__prefix+str(l)+'_n']
                n+=1
                #print n
        except KeyError:
            h="hi"
        #print n    
        
        if n==1:
            boop=[self.__prefix+'1_']
        else:
            boop=[self.__prefix+str(y)+'_' for y in range(1,n+1)]
            
        labels=list()
        for y in range(0,len(boop)):
            labels.append([boop[y]+str(x) for x in range(1,int(data[0][boop[y]+'n'])+1)])
        #print labels
        
        x=0
        grade_sum= [[0] * len(data) for p in range(n)]
        for k in range (0,len(data)):
            for i in range (0,len(labels)):
                for j in range (0,int(data[0][boop[i]+'n'])):
                    x+=float(data[k][labels[i][j]])
                    #print x
                grade_sum[i][k]=x
                x=0
            #a_student.add_grade(grade(prefix+"sum",value=grade_sum))
        return self.drop_lab(grade_sum,m)

In [21]:
dropped_labs=grade_summer_drop('l',data)######drops by lowest out of points given/total points in lab########
x=dropped_labs.drop_lab(Lab_grades,1)
print x,"\n\n"
x=dropped_labs.drop_lab(Lab_grades,2)
print x,"\n\n"


Dropped_labs_sum=list(dropped_labs.apply(1))
print Dropped_labs_sum




#copy=[[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
 #     [58.0, 0.0, 0.0, 62.0, 49.5, 69.0, 54.5, 69.0, 39.5, 60.0, 47.0, 68.5, 64.0, 49.5, 68.0, 59.0],
  #    [9.0, 0.0, 71.0, 102.0, 77.5, 89.0, 64.5, 40.0, 40.0, 26.0, 126.0, 97.5, 115.0, 114.5, 95.0, 106.0],
   #   [0.0, 0.0, 0.0, 75.0, 26.0, 35.0, 30.0, 51.0, 68.0, 27.0, 105.0, 51.0, 0.0, 25.0, 0.0, 53.0]]

[[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], [58.0, 'dropped', 'dropped', 62.0, 49.5, 69.0, 54.5, 69.0, 39.5, 60.0, 47.0, 68.5, 64.0, 49.5, 68.0, 59.0], [9.0, 0.0, 71.0, 102.0, 77.5, 89.0, 64.5, 40.0, 40.0, 26.0, 126.0, 97.5, 115.0, 114.5, 95.0, 106.0], ['dropped', 0.0, 0.0, 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped']] 


[[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], [58.0, 'dropped', 'dropped', 62.0, 49.5, 69.0, 54.5, 69.0, 39.5, 60.0, 47.0, 68.5, 64.0, 49.5, 68.0, 59.0], ['dropped', 'dropped', 71.0, 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped'], ['dropped', 0.0, 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'dropped', 'drop

*Exercise 7*: Write a new calculator that creates a new letter grade based on a weighted average of letter grades, by assigning the following numerical values to letter grades:

In [12]:
GradeMap={"A+":12,
          "A":11,
          "A-":10,
          "B+":9,
          "B":8,
          "B-":7,
          "C+":6,
          "C":5,
          "C-":4,
          "D+":3,
          "D":2,
          "D-":1,
          "F":0}
Weights={'Labs':10,
            'Homework':30,
            'Quiz':10,
            'Exam':30}

In [ ]:
class Weighted_Calculator():
    def __init__(self,prefix,weights):
        self.__prefix=prefix
        self.weights=weights
    def Calc_weight(self,LabsQuizExamMean=0):
        for i in range(LabsQuizExamMean):
            LabsQuizExamMean[i]=LabsQuizExamMean[i]/100.0
            
        

[10.0, 51.09375, 73.3125, 34.125]
[6.40625]
[1.0, 0.7299107142857143, 0.5236607142857143, 0.31022727272727274]


Test you calculator by applying the weights from the sy